In [57]:
import pandas
question = pandas.read_csv('./DeepLearningData.csv')
question = question.drop(columns=['index'])
question

,Unnamed: 0,Score,Title,Body,BestAns_RespTime_timestamp,FirstAns_RespTime_timestamp,Tag
0,0,26,7,293,2,2,3.0
1,1,144,7,24,9,0,4.0
2,2,21,3,54,7,7,3.0
3,3,53,5,41,5,1,4.0
4,4,49,6,169,2,0,4.0
...,...,...,...,...,...,...,...
1102523,1102523,0,5,63,0,0,2.0
1102524,1102524,0,13,51,0,0,3.0
1102525,1102525,0,11,586,1,1,3.0
1102526,1102526,-1,8,74,0,0,1.0


In [2]:
def to_binary(x):
    if x['FirstAns_RespTime_timestamp'] == 0:
        return 1
    else:
        return 0

question['FirstAns_RespTime_timestamp'] = question.apply(to_binary,axis=1)



In [58]:
question[question['FirstAns_RespTime_timestamp'] == 0]['Score'].count()

590675

In [59]:
question[question['BestAns_RespTime_timestamp'] == 1]['Score'].count()

107078

In [61]:
from sklearn.utils import shuffle
question_y_train = []
question_x_train = []

question_y_test = []
question_x_test = []
for i in range(10):
    cut = question[question['BestAns_RespTime_timestamp'] == i].sample(8471)
    question_y_train.append(cut[['BestAns_RespTime_timestamp']][847:])
    question_x_train.append(cut[['Tag','Score','Title','Body']][847:])
    question_y_test.append(cut[['BestAns_RespTime_timestamp']][:847])  
    question_x_test.append(cut[['Tag','Score','Title','Body']][:847])

question_x_train = pandas.concat(question_x_train)
question_y_train = pandas.concat(question_y_train)
question_x_test = pandas.concat(question_x_test)
question_y_test = pandas.concat(question_y_test)

question_x_train = shuffle(question_x_train)
question_y_train = shuffle(question_y_train)
question_x_test = shuffle(question_x_test)
question_y_test = shuffle(question_y_test)

def to_class(x):
    classes = []
    for i in range(10):
        classes.append(0)
    classes[x['BestAns_RespTime_timestamp']] = 1
    return classes

question_y_train = question_y_train.apply(to_class,axis=1)
question_y_test = question_y_test.apply(to_class,axis=1)

question_y_train = pandas.DataFrame(list(question_y_train),columns=list(range(0,10)))
question_y_test = pandas.DataFrame(list(question_y_test),columns=list(range(0,10)))

In [62]:
question_x_train = (question_x_train-question_x_train.mean())/question_x_train.std()
question_x_test = (question_x_train-question_x_train.mean())/question_x_train.std()

In [63]:
question_x_train

,Tag,Score,Title,Body
492607,1.605550,-0.152776,-0.125030,1.111910
247600,-1.703735,-0.152776,1.295157,-0.084133
9443,1.605550,-0.099339,-0.125030,-0.295884
283904,-1.703735,0.221282,-0.125030,1.177064
594045,-1.703735,-0.259649,1.011120,0.776832
...,...,...,...,...
153410,-0.876414,-0.152776,1.011120,-0.430846
959337,-1.703735,-0.045902,-0.125030,0.814063
7161,1.605550,0.114408,-0.977143,-0.307519
548899,-1.703735,-0.099339,-1.261180,-0.321480


In [86]:
import tensorflow.keras as keras
import tensorflow as tf

input_size = 4

model = keras.models.Sequential()
model.add(keras.layers.Input(shape = (input_size,)))
model.add(keras.layers.Dense(16, activation=tf.nn.relu))
model.add(keras.layers.Dense(32, activation=tf.nn.relu))
model.add(keras.layers.Dense(64, activation=tf.nn.relu))
model.add(keras.layers.Dense(32, activation=tf.nn.relu))
model.add(keras.layers.Dense(10,activation=tf.nn.softmax))

model.compile(optimizer=tf.optimizers.Adam(learning_rate=1e-4), loss='mse',metrics=['accuracy'])
model.fit(question_x_train, question_y_train, epochs=30, validation_split=0.2, batch_size=1024)

Epoch 1/30
60/60 [==============================] - 0s 6ms/step - loss: 0.0901 - accuracy: 0.1006 - val_loss: 0.0901 - val_accuracy: 0.1012
Epoch 2/30
60/60 [==============================] - 0s 5ms/step - loss: 0.0900 - accuracy: 0.1010 - val_loss: 0.0900 - val_accuracy: 0.1030
Epoch 3/30
60/60 [==============================] - 0s 5ms/step - loss: 0.0900 - accuracy: 0.1009 - val_loss: 0.0900 - val_accuracy: 0.1020
Epoch 4/30
60/60 [==============================] - 0s 5ms/step - loss: 0.0900 - accuracy: 0.1009 - val_loss: 0.0900 - val_accuracy: 0.1036
Epoch 5/30
60/60 [==============================] - 0s 5ms/step - loss: 0.0900 - accuracy: 0.1018 - val_loss: 0.0900 - val_accuracy: 0.1029
Epoch 6/30
60/60 [==============================] - 0s 5ms/step - loss: 0.0900 - accuracy: 0.1031 - val_loss: 0.0900 - val_accuracy: 0.1020
Epoch 7/30
60/60 [==============================] - 0s 5ms/step - loss: 0.0900 - accuracy: 0.1042 - val_loss: 0.0900 - val_accuracy: 0.1028
Epoch 8/30
60/60 [==

In [47]:
def myloss(y_true, y_pred):
    truelabel = tf.argmax(y_true,axis=0)
    predlabel = tf.argmax(y_pred,axis=0)
    return predlabel-truelabel

print(customMetric1([0,0,0],[1,0,0]))
print(myloss([0,1,0],[0,0,1]))

tf.Tensor(-0.33333334, shape=(), dtype=float32)
tf.Tensor(1, shape=(), dtype=int64)
tf.Tensor(1, shape=(), dtype=int64)
tf.Tensor(0, shape=(), dtype=int64)


In [84]:
out = model.predict_classes(question_x_test[-5:])
out

array([6, 6, 7, 4, 5])

In [85]:
question_y_test[-5:]

,0,1,2,3,4,5,6,7,8,9
8465,0,0,0,1,0,0,0,0,0,0
8466,0,0,1,0,0,0,0,0,0,0
8467,0,1,0,0,0,0,0,0,0,0
8468,0,0,0,0,0,0,0,0,1,0
8469,0,0,0,0,0,0,0,0,1,0


In [54]:
model.summary()

Model: "sequential_22"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_101 (Dense)            (None, 16)                80        
_________________________________________________________________
dense_102 (Dense)            (None, 32)                544       
_________________________________________________________________
dense_103 (Dense)            (None, 64)                2112      
_________________________________________________________________
dense_104 (Dense)            (None, 32)                2080      
_________________________________________________________________
dense_105 (Dense)            (None, 10)                330       
Total params: 5,146
Trainable params: 5,146
Non-trainable params: 0
_________________________________________________________________
